# Import and Setup

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [32]:
import pandas as pd
import numpy as np
from typing import Tuple

In [15]:
cols_to_use = ['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Arrest Type']

In [16]:
df = pd.read_csv("C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Traffic_Violations - Oct 6 2021.csv",
                nrows=5000,
                usecols=cols_to_use)

# Cleaning

## Assign Unique Stop ID 

In [17]:
def assign_stop_IDs(dataframe:pd.DataFrame) -> pd.DataFrame:
    """
    Assigns a unique ID # for each stop.
    Also deletes the 'SeqID' col.
    """
    dataframe['merged_id_col'] = dataframe['SeqID'] + ' _ ' + dataframe['Date Of Stop'] + ' _ ' + dataframe['Time Of Stop']
    
    unique_stops = dataframe['merged_id_col'].unique()
    
    stop_ID_dict = {stop_info:ID for ID, stop_info in enumerate(unique_stops)}

    dataframe.insert(loc=0,
                     column='Stop ID',
                     value=dataframe['merged_id_col'].map(stop_ID_dict))
    
    del dataframe['merged_id_col']
    del dataframe['SeqID']
    
    dataframe.set_index('Stop ID', inplace=True)

assign_stop_IDs(df)

## Convert strings to boolean int (0, 1)

In [18]:
def find_str_bool_cols(dataframe:pd.DataFrame) -> list:
    
    bool_cols = []
    
    str_bools_set = set(['Yes', 'No', np.nan])
    for col in df:
        if all([word in str_bools_set for word in df[col].unique()]):
            bool_cols.append(col)
            
    return bool_cols

def convert_str_bool_cols(dataframe:pd.DataFrame) -> pd.DataFrame:
    
    bool_cols = find_str_bool_cols(dataframe=dataframe)
    
    str_bool_mapping_dict = {'No':0, 'Yes':1}
    
    for col in bool_cols:
        dataframe[col] = dataframe[col].map(str_bool_mapping_dict)
    
convert_str_bool_cols(df)

## Create columns

In [35]:
def create_cols(dataframe:pd.DataFrame) -> pd.DataFrame:
    dataframe['Citation'] = (dataframe['Violation Type']=='Citation').astype(int)
    dataframe['Warning'] = (dataframe['Violation Type']=='Warning').astype(int)    
    
    dataframe['Male'] = (dataframe['Gender']=='M').astype(int)
    dataframe['Female'] = (dataframe['Gender']=='F').astype(int)
    
    dataframe['Probable Cause'] = (dataframe['Search Reason']=='Probable Cause').astype(int)
    
    dataframe['DateTime'] = pd.to_datetime(df['Date Of Stop'] + ' ' + df['Time Of Stop'])
    del dataframe['Date Of Stop'], dataframe['Time Of Stop']

create_cols(df)

# Speed Columns

In [20]:
def get_speed_and_limit_from_split_str(split_description:list) -> Tuple[float, float]:
    """
    :INPUT:
    'split_description'=df['Description].str.split() element

    :OUTPUT:
    [posted_limit, speed_over_limit]
    speed_over_limit is often not recorded, so this sometimes returns [posted_limit, np.nan]
    """
    speeds_from_description = tuple([int(word) for word in split_description if word.isdigit()])
    
    if len(speeds_from_description)==2: # if posted limit and driver's speed are both recorded
        return sorted(speeds_from_description)
    elif len(speeds_from_description)==1: # if only posted limit is recorded
        return (speeds_from_description[0], np.nan)
    elif len(speeds_from_description)==0: # if neither posted limit nor driver speed were recorded
        return (np.nan, np.nan)
    

In [21]:
def create_speed_columns(dataframe:pd.DataFrame) -> pd.DataFrame:
    limit_and_speed_cols = dataframe['Description'].str.split().apply(get_speed_and_limit_from_split_str).apply(pd.Series)
    
    limit_and_speed_cols.rename(columns={0:'Speed Limit',
                                         1:'Recorded Speed'}, inplace=True)
    
    return pd.concat([dataframe, limit_and_speed_cols], axis=1)
    
df = create_speed_columns(df)

# Export

In [33]:
df.to_csv("C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021.csv")

In [36]:
df

,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,...,Gender,Arrest Type,Citation,Warning,Male,Female,Probable Cause,Speed Limit,Recorded Speed,DateTime
Stop ID,,,,,,,,,,,,,,,,,,,,,
0,MCP,"2nd District, Bethesda","LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",CORDELL ST @ NORFOLK AVE.,38.989743,-77.097770,0,0,0,0,...,M,A - Marked Patrol,1,0,1,0,0,NaN,NaN,2019-08-11 20:02:00
1,MCP,"2nd District, Bethesda",EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,NBI270 AT MIDDLEBROOK RD,39.174110,-77.246170,0,0,0,0,...,M,A - Marked Patrol,1,0,1,0,0,55.0,85.0,2019-08-12 13:41:00
2,MCP,"5th District, Germantown",DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,MIDDLEBROOK AN 355,39.182016,-77.238221,0,0,0,0,...,M,A - Marked Patrol,1,0,1,0,0,NaN,NaN,2019-08-12 21:00:00
3,MCP,"5th District, Germantown",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,GERMANTOWN RD AND ALE HOUSE,39.160508,-77.284023,0,0,0,0,...,M,A - Marked Patrol,1,0,1,0,0,NaN,NaN,2019-08-12 21:43:00
4,MCP,"2nd District, Bethesda",FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,EASTWEST/ 355,38.984247,-77.090548,0,0,0,0,...,M,A - Marked Patrol,1,0,1,0,0,30.0,NaN,2019-08-12 21:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,MCP,"3rd District, Silver Spring",DRIVING VEH. ON HWY. WITH UNPAID REGISTRATION FEE,SB COLESVILLE ROAD @ NORTH NOYES DRIVE,39.001677,-77.026020,0,0,0,0,...,M,A - Marked Patrol,0,1,1,0,0,NaN,NaN,2020-03-06 07:51:00
2820,MCP,"3rd District, Silver Spring",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,SB COLESVILLE ROAD @ NORTH NOYES DRIVE,39.001677,-77.026020,0,0,0,0,...,M,A - Marked Patrol,0,1,1,0,0,NaN,NaN,2020-03-06 07:51:00
2821,MCP,"3rd District, Silver Spring",DISPLAYING EXPIRED REGISTRATION PLATE ISSUED B...,NB NEW HAMPSHIRE AVE @ QUAINT ACRES DRIV,39.048562,-76.992987,0,0,0,0,...,F,A - Marked Patrol,0,1,0,1,0,NaN,NaN,2020-03-06 13:00:00
